# SOCIAL DITANCING AND FACE MASK DETECTION

In [1]:
import cv2
import cvzone
import math
import numpy as np
import os
import matplotlib.pyplot as plt
from cvzone.FaceMeshModule import FaceMeshDetector

## 1.1 FUNCTION TO CALCULATE THE DISTANCE FROM THE CAMERA

In [2]:
import math
from itertools import combinations

def detect_distance_from_camera(img, detector):
    #detect the faceMesh
    img, faces = detector.findFaceMesh(img, draw=False)
    
    distances = list()
    
    if faces:
        for face in faces:
            pointLeft = face[145]
            pointRight = face[374]
            
            color = (0, 200, 0)
            #cv2.line(img, pointLeft, pointRight, color, 3)
            #cv2.circle(img, pointLeft, 5, (255,0,255), cv2.FILLED)
            #cv2.circle(img, pointRight, 5, (255,0,255), cv2.FILLED)
            
            w = detector.findDistance(pointLeft, pointRight)[0]
            #cv2.putText(img, str(w[0]), (pointLeft), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            
            # Trying to find the deep of the focal lent
            W = 6.3 #avergae distance between the eyes
            #d = 30
            #f = (w*d)/W
            f = 540
            #cv2.putText(img, str(f), (pointLeft),cv2.FONT_HERSHEY_SIMPLEX, 3, (0,0,0), 2)
            
            # Finding the distance from the focal point
            d = (W*f)/w
            
            # Create a list with the coordinates of the eyes and the distance
            distances.append([d, pointLeft, pointRight])
            #cv2.putText(img, "{:.2f} cm".format(d), (face[10]),cv2.FONT_HERSHEY_SIMPLEX, 2/int(d*0.047), (0,0,0), 2)
            
    return distances

## 1.2 Person detection with opencv

In [3]:
# load our serialized person detector model from disk
print("[INFO] loading person detector model...")
prototxtPath = os.path.sep.join(["person_detector", "SSD_MobileNet_prototxt.txt"])
weightsPath = os.path.sep.join(["person_detector",
                                "SSD_MobileNet.caffemodel"])
net_person = cv2.dnn.readNet(prototxtPath, weightsPath)

[INFO] loading person detector model...


In [4]:
import cv2
import time
import imutils
import argparse
import numpy as np

from imutils.video import FPS
from imutils.video import VideoStream

#Initialize Objects and corresponding colors which the model can detect
labels = ["background", "aeroplane", "bicycle", "bird", 
"boat","bottle", "bus", "car", "cat", "chair", "cow", 
"diningtable","dog", "horse", "motorbike", "person", "pottedplant", 
"sheep","sofa", "train", "tvmonitor"]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

#Loading Caffe Model
print('[Status] Loading Model...')
prototxtPath = os.path.sep.join(["person_detector", "SSD_MobileNet_prototxt.txt"])
weightsPath = os.path.sep.join(["person_detector",
                                "SSD_MobileNet.caffemodel"])
nn = cv2.dnn.readNet(prototxtPath, weightsPath)

img = cv2.imread('images/pic2.jpg')

img = imutils.resize(img, width=400)
(h, w) = img.shape[:2]

#Converting Frame to Blob
blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 
    0.007843, (300, 300), 127.5)

#Passing Blob through network to detect and predict
nn.setInput(blob)
detections = nn.forward()

[Status] Loading Model...


In [5]:
#Loop over the detections
for i in np.arange(0, detections.shape[2]):

#Extracting the confidence of predictions
    confidence = detections[0, 0, i, 2]

    #Filtering out weak predictions
    if confidence > 0.7:
            
        #Extracting the index of the labels from the detection
        #Computing the (x,y) - coordinates of the bounding box        
        idx = int(detections[0, 0, i, 1])

        #Extracting bounding box coordinates
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")

        #Drawing the prediction and bounding box
        label = "{}: {:.2f}%".format(labels[idx], confidence * 100)
        cv2.rectangle(img, (startX, startY), (endX, endY), colors[idx], 2)

        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(img, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)

plt.imshow(img)

NameError: name 'plt' is not defined

In [6]:
#Initialize Video Stream
print('[Status] Starting Video Stream...')
vs = VideoStream(src=0).start()
time.sleep(2.0)
fps = FPS().start()

#Loop Video Stream
while True:

    #Resize Frame to 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)
    (h, w) = frame.shape[:2]

    #Converting Frame to Blob
    blob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 
    	0.007843, (300, 300), 127.5)

    #Passing Blob through network to detect and predict
    nn.setInput(blob)
    detections = nn.forward()


    #Loop over the detections
    for i in np.arange(0, detections.shape[2]):

	#Extracting the confidence of predictions
        confidence = detections[0, 0, i, 2]

        #Filtering out weak predictions
        if confidence > 0.7:
            
            #Extracting the index of the labels from the detection
            #Computing the (x,y) - coordinates of the bounding box        
            idx = int(detections[0, 0, i, 1])

            #Extracting bounding box coordinates
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            #Drawing the prediction and bounding box
            label = "{}: {:.2f}%".format(labels[idx], confidence * 100)
            cv2.rectangle(frame, (startX, startY), (endX, endY), colors[idx], 2)

            y = startY - 15 if startY - 15 > 15 else startY + 15
            cv2.putText(frame, label, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[idx], 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    
    fps.update()

fps.stop()

print("[Info] Elapsed time: {:.2f}".format(fps.elapsed()))
print("[Info] Approximate FPS:  {:.2f}".format(fps.fps()))

cv2.destroyAllWindows()
vs.stop()

[Status] Starting Video Stream...
[Info] Elapsed time: 3.10
[Info] Approximate FPS:  24.22


In [14]:
cv2.destroyAllWindows()


In [6]:
import torch

#model = torch.hub.load('ultralytics/yolov5', 'custom', 'yolov5s-fp16.tflite')
yolo_model = torch.hub.load("ultralytics/yolov5", "yolov5s") 

def euclidean_dist(p1, p2):
    """
    p1, p2 = two points for calculating Euclidean Distance

    :return:
    dst = Euclidean Distance between two 2d points
    """
    if p1 is int:
        dst = math.sqrt(p1 ** 2 + p2 ** 2)
        print(type(p1))
    else:
        dst = math.sqrt((p2[0] - p1[0] ) ** 2 + (p2[1] - p1[1])** 2)
    
    return dst
        


def detect_sd(frame, distances):
    detections = yolo_model(frame, size=640)
    detections = detections.pandas().xyxy[0].to_dict(orient="records")
    print(detections)

    if len(detections) > 0:  # At least 1 detection in the image and check detection presence in a frame
        centroid_dict = dict()  # Function creates a dictionary and calls it centroid_dict
        distances_dict = dict()
        objectId = 0  # We inialize a variable called ObjectId and set it to 0
        for detection in detections:  # In this if statement, we filter all the detections for persons only
            # Check for the only person name tag 
            name_tag = detection['name']  # Coco file has string of all the names
            if name_tag == 'person':
                xmin, ymin, xmax, ymax = detection['xmin'], detection['ymin'], detection['xmax'], detection[
                    'ymax']  # Store the center points of the detections
                x = int(xmin + (xmax - xmin) / 2)
                y = int(ymin + (ymax - ymin) / 2)
                centroid_dict[objectId] = (x, y, int(xmin), int(ymin), int(xmax),
                                           int(ymax))  # Create dictionary of tuple with 'objectId' as the index center points and bbox
                #[for distance in distances]
                distances_dict[objectId] = [distance for distance in distances if xmin < distance[1][0] < xmax and ymin < distance[1][0] < ymax]
                objectId += 1  # Increment the index for each detection


        # =================================================================#

        # =================================================================
        # 3.2 Purpose : Determine which person bbox are close to each other
        # =================================================================
        print('Eye distances: ', distances_dict)
        print('Persons recogized: ', centroid_dict)
        red_zone_list = []  # List containing which Object id is in under threshold distance condition.
        red_line_list = []
        for (id1, p1), (id2, p2) in combinations(centroid_dict.items(),2):  # Get all the combinations of close detections, #List of multiple items - id1 1, points 2, 1,3
            dx, dy = p1[0] - p2[0], p1[1] - p2[1]  # Check the difference between centroid x: 0, y :1
            socialdist_px = euclidean_dist(p1[0:2], p2[0:2])  # Calculates the Euclidean distance
                
            socialdist_px = euclidean_dist(p1, p2)
            distance_bepx1 = 0
            distance_bepx2 = 0
            no_eye_ref = False
            
            distance_fc1 = 0
            distance_fc2 = 0
            
            if id1 in distances:
                if len(distances[id1]) > 0:
                    distance_bepx1 = euclidean_dist(distances[id1][0][1], distances[id1][0][2])
                    distance_fc1 = distances[id1][0][0]
            if id2 in distances:
                if len(distances[id2]) > 0:
                    distance_bepx2 = euclidean_dist(distances[id2][0][1], distances[id2][0][2])
                    distance_fc1 = distances[id2][0][0]
            
            if distance_bepx1 > distance_bepx2:
                socialdist = socialdist_px*6.3/distance_bepx1                    
            elif distance_bepx1 < distance_bepx2:
                socialdist = socialdist_px*6.3/distance_bepx2
            else:
                socialdist = socialdist_px
                no_eye_ref = True
                
            if distance_fc1 > 0 and distance_fc2 > 0:
                if distance_fc2 > distance_fc1:
                    socialdist = math.sqrt((distance_fc2-distance_fc1)**2 + (socialdist)**2)
            
            ### Referencing horizontal distances to between eyes distance (6.3cm)
            if (socialdist < 150.0 and not no_eye_ref) or (socialdist < 300 and no_eye_ref):  # Set our social distance threshold - If they meet this condition then..
                if id1 not in red_zone_list:
                    red_zone_list.append(id1)  # Add Id to a list
                    red_line_list.append(p1[0:2])  # Add points to the list
                    #red_zone_dists.append(socialdist)
                if id2 not in red_zone_list:
                    red_zone_list.append(id2)  # Same for the second id
                    red_line_list.append(p2[0:2])
                ###############
                # Print the distance horizontal in cm
                ##############
                cv2.line(frame, p1[0:2], p2[0:2], (0, 0, 255), 1)
                if not no_eye_ref:
                    cv2.putText(frame, "{:.2f} cm".format(socialdist),(p1[2],p1[3]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)  # Display Text
                ################
            print(p1, '####', p2, '####', socialdist)
        for idx, box in centroid_dict.items():  # dict (1(key):red(value), 2 blue)  idx - key  box - value
            if idx in red_zone_list:  # if id is in red zone list
                #cv2.putText(frame, "{:.2f} cm".format(socialdist),(box[2], box[3]),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
                cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (0, 0, 255),
                              2)  # Create Red bounding boxes  #starting point, ending point size of 2
            else:
                cv2.rectangle(frame, (box[2], box[3]), (box[4], box[5]), (0, 255, 0),
                              2)  # Create Green bounding boxe bounding boxes  #starting
        # =================================================================
        # 3.3 Purpose : Display Risk Analytics and Show Risk Indicators
        # =================================================================
        text = "People at Risk: %s" % str(len(red_zone_list))  # Count People at Risk
        location = (10, 25)  # Set the location of the displayed text
        cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1,
                    cv2.LINE_AA)  # Display Text

        for check in range(0,
                           len(red_line_list) - 1):  # Draw line between nearby bboxes iterate through redlist items
            start_point = red_line_list[check]
            end_point = red_line_list[check + 1]
            check_line_x = abs(end_point[0] - start_point[0])  # Calculate the line coordinates for x
            check_line_y = abs(end_point[1] - start_point[1])  # Calculate the line coordinates for y
            if (check_line_x < 75) and (
                    check_line_y < 25):  # If both are We check that the lines are below our threshold distance.
                cv2.line(frame, start_point, end_point, (255, 0, 0),
                         2)  # Only above the threshold lines are displayed.
    # =================================================================#
    return frame

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

# Face mask detection

In [7]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join(["face_detector", "deploy.prototxt"])
weightsPath = os.path.sep.join(["face_detector",
                                "res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)
model = load_model("mask_detector.model")

2022-02-11 23:08:17.208726: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jejobueno/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-11 23:08:17.208752: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[INFO] loading face detector model...


2022-02-11 23:08:18.397251: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/jejobueno/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-02-11 23:08:18.397283: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-11 23:08:18.397308: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jejobueno-HP-EliteBook-840-G2): /proc/driver/nvidia/version does not exist
2022-02-11 23:08:18.397527: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-

In [8]:
def draw_detections(detections, image, w, h):
    masks = 0
    # loop over the detections
    for i in range(0, detections.shape[2]):
        # extract the confidence (i.e., probability) associated with
        # the detection
        confidence = detections[0, 0, i, 2]

        # filter out weak detections by ensuring the confidence is
        # greater than the minimum confidence
        if confidence > 0.5:
            # compute the (x, y)-coordinates of the bounding box for
            # the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # ensure the bounding boxes fall within the dimensions of
            # the frame
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # extract the face ROI, convert it from BGR to RGB channel
            # ordering, resize it to 224x224, and preprocess it
            face = image[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # pass the face through the model to determine if the face
            # has a mask or not
            (mask, withoutMask) = model.predict(face)[0]

            # determine the class label and color we'll use to draw
            # the bounding box and text
            label = "Mask" if mask > withoutMask else "No Mask"
            masks = masks + 0 if mask > withoutMask else masks + 1
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)

            # display the label and bounding box rectangle on the output
            # frame
            cv2.putText(image, label, (startX, startY - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
            
    text = "People awithout mask: %s" % str(masks)  # Count People at Risk
    location = (10, 50)  # Set the location of the displayed text
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1,
            cv2.LINE_AA)  # Display Text
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def detect(image):
    (h, w) = image.shape[:2]

    # construct a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
                                 (104.0, 177.0, 123.0))

    # pass the blob through the network and obtain the face detections
    #print("[INFO] computing face detections...")
    net.setInput(blob)
    detections = net.forward()
    
    #drawing boundry boxes
    detected_image = draw_detections(detections, image, w, h)
    return detected_image

# Detect with camera

In [9]:
import io
import traceback

import cv2
import cvzone
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

cap = cv2.VideoCapture(0)
detector = FaceMeshDetector()
cv2.namedWindow("Image", cv2.WINDOW_NORMAL)

try:
    while True:
        success, frame = cap.read()

        #########################################
        # Face and distance from screen detection
        #########################################

        distances = detect_distance_from_camera(frame, detector)

        #######################################
        #    People detection:
        #######################################

        frame = detect_sd(frame, distances)
        
        #######################################
        #    Face Mask Detection
        ########################################
        
        # Draw the detected objects
        detected_frame = detect(frame)
        detected_frame = cv2.cvtColor(detected_frame, cv2.COLOR_RGB2BGR)
        
        ########################################
        # Show on screen
        ########################################
        cv2.resizeWindow('Image', 600,600)
        cv2.imshow("Image", detected_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            cap.release()
            break
except Exception as errors:
    errors = io.StringIO()
    traceback.print_exc(file=errors)
    contents = str(errors.getvalue())
    print(contents)
    errors.close()
finally:
    cv2.destroyAllWindows()
    cap.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Traceback (most recent call last):
  File "/tmp/ipykernel_46486/3537944629.py", line 27, in <module>
    frame = detect_sd(frame, distances)
NameError: name 'detect_sd' is not defined

